In [15]:
from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig, AutoModelForSeq2SeqLM, AutoConfig

'''
T5 계열 중 사용 가능한 모델 예시
- csebuetnlp/mT5_multilingual_XLSum
- eenzeenee/t5-base-korean-summarization
...

BART 계열 중 사용 가능한 모델 예시
- EbanLee/kobart-summary-v3
- digit82/kobart-summarization
...
'''

model_name = "eenzeenee/t5-small-korean-summarization" 

tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [16]:
model_config = AutoConfig.from_pretrained(model_name)
print(f"{model_config.num_beams}\n"
f"{model_config.max_length}\n"
f"{model_config.no_repeat_ngram_size}\n"
f"{model_config.length_penalty}")

config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

1
128
0
1.0


In [17]:
tokenizer.tokenize("안녕")

['ìķĪëħķ']

- ⚠️ config의 inference/remove_tokens에는 Special_token을 추가하지 전에 토크나이저가 기본적으로 갖고 있는 special tokens를 설정해줘야 한다.

In [23]:
import collections.abc
def flatten(l):
    for el in l:
        if isinstance(el, collections.abc.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el
try:
    remove_tokens = list(flatten(tokenizer.special_tokens_map.values()))
except:
    remove_tokens = []
print(remove_tokens)

['</s>', '<pad>', '<pad>', '<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_id_53>', '<extra_id_54>', '<extra_id_55>', '<extra_id_56>', '<extra_id_57>

- ⚠️ 만약 bos_token이 None이라면 토크나이저에 맞는 처리를 해야 한다.
    - T5 계열 모델의 토크나이저는 bos_token을 사용하지 않는다. 그냥 None으로 놔두면 된다.

In [5]:
'''
    BART Embedding Layer: {model.get_encoder().embed_tokens}, {model.get_encoder().embed_positions}
    tokenizer의 encoder_max_len은 {model.get_encoder().embed_positions.num_embeddings} 이하여야 한다.

    BART Decoder Layer: {model.get_decoder().embed_tokens}, {model.get_decoder().embed_positions}
    tokenizer의 decoder_max_len은 {model.get_decoder().embed_positions.num_embeddings} 이하여야 한다.
'''

# from pprint import pprint
print(f'''
    BOS token id : {tokenizer.bos_token_id},
    BOS : {tokenizer.bos_token},
    EOS : {tokenizer.eos_token},
    PAD : {tokenizer.pad_token}
    Special_tokens : {tokenizer.special_tokens_map},
    Tokenizer's max_model_input_sizes : {tokenizer.model_max_length}

''')


    BOS token id : None,
    BOS : None,
    EOS : </s>,
    PAD : <pad>
    Special_tokens : {'eos_token': '</s>', 'unk_token': '<pad>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<e

In [6]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
project_dir = "/data/ephemeral/home/nlp-5/auto1p/"
import os
import sys
sys.path.append(
    project_dir
)
from src.utils.utils import *

In [7]:
current_time = get_current_time()
concept = "t5" # 실험 컨셉을 작성. output 디렉토리 이름 및 config 파일 이름 설정 시 반영됨.
output_dir = f"./outputs/exp_{concept}_{current_time}"
print(current_time)

08020735


In [8]:
from src.utils.eda_length import *


In [9]:
save_eval_log_steps = 1 # save_step은 eval, log step의 배수여야 한다. 같이 맞춰주는 것이 편하다.
train_batch_size = 8
inference_batch_size = 8 # eval batch size 도 동일하게 설정됨
encoder_max_len = 1300 # 모델의 encoder input length 보다는 작게 설정해야 함. (확인하는 방법이 모델마다 다름;;)
decoder_max_len = 170 # inference max length 도 동일하게 설정됨 > 170 이하를 추천.

In [10]:
config_data = {
    "general": {
        "data_path": "./data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": model_name, # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": output_dir, # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
        "train_data": "train.csv", # 학습 데이터 파일을 설정한다. data_path 폴더 안에 존재해야 한다.
        "val_data": "dev.csv",
        "test_data": "test.csv", # 추론 데이터 파일을 설정한다. 추론 데이터에 topic이나 ner을 추가할 경우에 대비.
        "eval_tokenizer": "upstage/solar-pro2-tokenizer",
    },
    "tokenizer": {
        "encoder_max_len": encoder_max_len,
        "decoder_max_len": decoder_max_len,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": [ '#Person1#', '#Person2#', '#Person3#', '#Person4#',
            '#Person5#', '#Person6#', '#Person7#',
            '#PhoneNumber#', '#Address#', '#DateOfBirth#','#PassportNumber#','#SSN#','#CardNumber#','#CarNumber#','#Email#',
            # '#Topic#','#Dialogue#','#SEP#',
        ]
    },
    "training": {
        "seed": 42,
        "output_dir": output_dir,
        "overwrite_output_dir": False,

        "save_total_limit": 1,
        "load_best_model_at_end": True,
        "save_steps": save_eval_log_steps,

        "logging_dir": output_dir,
        "logging_steps": save_eval_log_steps,

        "num_train_epochs": 5,
        "per_device_train_batch_size": train_batch_size,
        "remove_unused_columns": True,
        "fp16": False, # float16 사용 메모리 절약, But 정밀도 문제 존재 > 구형 GPU에서 사용
        "bf16": True, # float16 사용 메모리 절약, 정밀도 문제 개선 > 30/40,... 등 최신 GPU 가능
        "dataloader_drop_last": False,
        "group_by_length": True,
        
        "gradient_checkpointing": True,
        "gradient_checkpointing_kwargs": {"use_reentrant": False, 'use_cache': False},
        "gradient_accumulation_steps": 1,
        "torch_empty_cache_steps": 1,
        "dataloader_num_workers": 8,

        "per_device_eval_batch_size": inference_batch_size,
        "evaluation_strategy": 'steps',
        "eval_steps": save_eval_log_steps,
        
        "predict_with_generate": True,
        "generation_max_length": decoder_max_len,
        
        # Callbacks
        "early_stopping_patience": 2,
        "early_stopping_threshold": 0.001,

        # Optimizer
        "learning_rate": 1e-5,
        "warmup_steps": 10,
        "weight_decay": 1e-3,
        "num_cycles": 1,

        "report_to": "none", # 학습 과정을 어느 백엔드에 저장할 것인지. "none" 설정 시 wandb 미사용. "wandb" 설정 시 사용.

        # PEFT
        "LoRA": False,
        "QLoRA": False,
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "skiersong", # 팀 실험 시 organization 이름
        "project": "nlp-5-upgrade",
        "name": f"a_base_{current_time}", # 개별 실험 이름
        # "group": "", # 유사한 실험들은 같은 그룹으로 설정
        "notes": "AutoModel baseline", # 실험에 대한 추가 설명
    },
    "inference": {
        "ckt_dir": os.path.join(output_dir, 'best'), # 파인튜닝이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": os.path.join(output_dir, f"submission_{current_time}.csv"), # 제출할 csv 파일 저장 경로
        "no_repeat_ngram_size": 2, # 
        "early_stopping": True,
        "generate_max_length": decoder_max_len,
        "num_beams": 4,
        "batch_size" : inference_batch_size,
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": remove_tokens,
        "length_penalty": 1.2 # > 1로 설정하면 짧은 문장을 선호하도록 페널티를 강화, 1보다 작은 값을 주면 긴 생성문을 선호하게 된다.
    }
}

In [11]:
import os
import yaml
from pprint import pprint

config_path = os.path.join(
    project_dir,'src','configs',
    f"config_{concept}_{current_time}.yaml" # config 파일 이름을 설정
)
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': './data/',
             'eval_tokenizer': 'upstage/solar-pro2-tokenizer',
             'model_name': 'eenzeenee/t5-base-korean-summarization',
             'output_dir': './outputs/exp_t5_08020735',
             'test_data': 'test.csv',
             'train_data': 'train.csv',
             'val_data': 'dev.csv'},
 'inference': {'batch_size': 8,
               'ckt_dir': './outputs/exp_t5_08020735/best',
               'early_stopping': True,
               'generate_max_length': 170,
               'length_penalty': 1.2,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['</s>',
                                 '<pad>',
                                 '<pad>',
                                 ['<extra_id_0>',
                                  '<extra_id_1>',
                                  '<extra_id_2>',
                                  '<extra_id_3>',
                                  '<extra_id_4>',
